# Bioinformatics Group Project

This project will focus on analyzing simulated RNAseq data in order to evaluate differential gene expression between control and treatment samples. The goal is to combine your knowledge of Unix, Python/R, and bioinformatic tools muscle and hmmer to “replicate” a recent study of gene expression in kidney tissues of normal (control) and obese mice (Kuhns & Pluznick 2017). In this project, we have approximated this process by building protein models of 6 target transcripts, searching simulated transcriptome data for “hits,” and counted those hits as a proxy for gene expression.


### Part 1: Use BLAST to identify the genes encoded by the 6 differentially expressed transcripts listed in *uniquetranscripts.fasta*

* **Step 1**: Go to the NCBI Blast website
* **Step 2**: Do a megablast search of the fasta sequences in the *uniquetranscripts.fasta* file. Make sure to set the database as "nucleotide collection".
* **Step 3**: Download results of each transcript blasts as a CSV Hit Table
* **Step 4**: In unix, use a code to compile all the top hits for the transcripts(should be the 1st row in the table)

In [ ]:
for file in *.csv; do head -1 $file; done > topHits.csv

# Works only when the data is in multiple csv files
 
#If you didn't want to use a for loop then you could do each file separately and then append them to the same document
#Example code: head -1 filename.csv >> output.csv

### Part 2: Search the NCBI protein database for amino acid sequences corresponding to these 6 transcripts

* **Step 1**: Go to the NCBIprotein database
* **Step 2**: Based on the top hits from part one and the protein names associated with them, search the protein data base for sequences.
* **Step 3**: Choose 10-20 protein sequences from mice (Mus musculus) and other closely related organisms
* **Step 4**: Download selected sequences saving one fasta file of protein sequences per transcript

### Part 3: Write a python or R script to translate nucleotide sequences into amino acid sequences

#### Basic Pseudocode: 
1. Import packages
2. Import codon table as a dataframe while creating a dictionary for the codon map. 
    * Doing so avoids lots of issues down the road
3. Create a for loop to read the codonmap.txt file into the codonMapDict dictionary. 
    * Reversing the columns helps with completing the dictionary; accomplished by an if statement.	
4. Make a definition for translating DNA, including a sequence searcher, the codonMapDict from above, and the stop codons as a variable of either TAA, TAG, or TGA. 
    * Regex can be a bit greedy if not properly programmed, due to tryptophan having TGG codon. We avoided this by assigning the three stop codons specifically.
5. After the definition, simply start the sequence searcher to find ATG.
6. Trim out the sequences up to the first start codon (ATG) then split the sequence into codons (increments of three).
7. Sequentially build four different commands based on outfile to make an outfile (for example, control1Protein) as a writable file.
	* Do this for all files in question.
8. Close codons at the end with codons.close()

In [ ]:
# Import packages
import re
import sys
import itertools
from sys import argv
from itertools import takewhile

# Open data files 
codons = open ("codonmap.txt", 'r')

# Create a dictionary for codon map 
codonMapDict = {}

# Create for loop to read codonmap.txt into dictionary (reversed columns 0 and 1 to create full dictionary)
for line in codons:
    line = line.strip()
    cols = line.split()
    if cols[1] in codonMapDict:
        print ("Duplicate: " + codonMapDict [0])
        break
    else:
        codonMapDict [cols[1]] = cols[0]

#Create definition to be used in subsequent file conversions
def translateDna(sequence, codonMapDict, stopCodons = ('TAA', 'TGA', 'TAG')):
    #find first start codon 
    start = sequence.find('ATG')    
    #trim sequence starting at first start codon 
    trimmedSequence = sequence[start:]    
    #split sequence into codons 
    codons = [trimmedSequence[i:i+3] for i in range(0, len(trimmedSequence), 3)]
    codingSequence = takewhile(lambda x: x not in stopCodons and len(x) == 3 , codons)
    proteinSequence = ''.join([codonMapDict[codon] for codon in codingSequence])

    return "{0}_".format(proteinSequence)

# File 1
outfile = open ("control1Protein", 'w')
for line in open("control1.fasta", 'r'):
    line = line.strip()
    if ">" in line: 
        header = line 
        outfile.write(header + "\n")
    else:
        sequence = line 
        x = translateDna (sequence, codonMapDict, stopCodons = ('TAA', 'TGA', 'TAG')) 
        outfile.write(x + "\n")
outfile.close()

# File 2
outfile = open ("control2Protein", 'w')
for line in open("control2.fasta", 'r'):
    line = line.strip()
    if ">" in line: 
        header = line 
        outfile.write(header + "\n")
    else:
        sequence = line 
        x = translateDna (sequence, codonMapDict, stopCodons = ('TAA', 'TGA', 'TAG')) 
        outfile.write(x + "\n")
outfile.close()

# File 3
outfile = open ("obese1Protein", 'w')
for line in open("obese1.fasta", 'r'):
    line = line.strip()
    if ">" in line: 
        header = line 
        outfile.write(header + "\n")
    else:
        sequence = line 
        x = translateDna (sequence, codonMapDict, stopCodons = ('TAA', 'TGA', 'TAG')) 
        outfile.write(x + "\n")
outfile.close()

#File 4
outfile = open ("obese2Protein", 'w')
for line in open("obese2.fasta", 'r'):
    line = line.strip()
    if ">" in line: 
        header = line 
        outfile.write(header + "\n")
    else:
        sequence = line 
        x = translateDna (sequence, codonMapDict, stopCodons = ('TAA', 'TGA', 'TAG')) 
        outfile.write(x + "\n")
outfile.close()

codons.close()

### Part 4: Build a Hidden Markov Model for each of the 6 transcript proteins and search the 4 translated “RNAseq files”

#### Basic Pseudocode
1. Do a loop for muscle alignments for each of the 6 transcripts from the downloaded protein sequences
2. Do a HMM build in a loop using the new alignments and 6 
3. Do a HMM search using the HMMbuild files and the fasta files containing sequences to be searched

In [ ]:
# Step 1
for i in *.fasta
do
../muscle3.8.31_i86win32.exe -in $i -out $i.align
done

# Step 2
for i in *.align
do
../hmmer-3.1b2-cygwin64/binaries/hmmbuild.exe $i.hmm $i
done

# Step 3
for i in *.fasta
do
../hmmer-3.1b2-cygwin64/binaries/hmmsearch.exe --tblout $i.hits (loop all .hmm files) $i
done

### Part 5: Graph the “expression levels” of each protein in each of the “RNAseq files.”

#### Basic Pseudocode
1. Import packages, including numpy, pandas, scipy, and plotnine
2. Establish RNAseq files as a dataframes (control1.fasta=1, control2.fasta=2, obese1.fasta=3, and obese2.fasta=4)
3. From the dataframes, write a ggplot script for a violin graph with the following conditions...
    * Points on the x axis belong to the transcript names (x=[1], for example)
	* The y axis spans a numerical range based on the number of transcripts
	* The dataframes are all plotted on one graph, with each dataframe having a separate color
        * The violin graph should accomodate the entire range of transcript levels per each transcript, while allowing for all transcripts to be readily discernable.

In [ ]:
#import packages
import numpy
import pandas
import scipy
from plotnine import *

#assign ProteinCounts CSV as dataFrame
ProteinDF=pandas.read_csv("ProteinCounts.csv",header=0)
#make shape so that ggplot can read file
ProteinDF.shape
a=ggplot(ProteinDF,aes(x="Protein",y="Control1"))+geom_point(color="blue")
a+theme_classic()
b=ggplot(ProteinDF,aes(x="Protein",y="Control2"))+geom_point(color="red")
b+theme_classic()
c=ggplot(ProteinDF,aes(x="Protein",y="Obese1"))+geom_point(color="green")
c+theme_classic()
d=ggplot(ProteinDF,aes(x="Protein",y="Obese2"))+geom_point(color="black")
d+theme_classic()

The results from the manuscript indicate that there are 4 genes that are upregulated and 2 that are downregulated.
* Upregulated Genes
    * Synpr
    * Lhx2
    * Atp12a
    * Ptpn5
* Downregulated Genes
    * Gsta2
    * Slc7a12


From our experimental results, we obtained the following up- and downregulated genes...

* Upregulated Genes
    * Atp12a
    * Synpr
* Downregulated Genes
    * Gsta2
    * Slc7a12
* No Data on these Genes
    * Lhx2
    * Ptpn5

Lhx2 and Ptpn5 could have been resulted in no data due to no sequences being found in our experiment files.

## Further Explorations 

### Question 1: 

Search resticted to Mouse

* Discontinuous MegaBlast
    * Looks for longer matches therefore has less 'hits' (for example transcript 2 has 44 hits compared to the 123 hits from using Blastn) (146 vs 169 transcript 6) (262 vs 295 transcript 8)
    * Accounts for genomes that have gapped alignments. Thus aligning sequences containing regions that don't have a match to the query sequence but still match portions of the query. Would probably be best on different species instead of the same type of species(i.e. mouse/Mus).
* Blastn
    * Many more hits found here -- not as specific sequences are included
    * Another indication of the less specific results given from a Blastn search is the E value accepted values. For example the highest E value found under a discontinuous MegaBlast is 1.2 while 4.2 is found under the Blastn E values for transcript 8.

### Question 2:

In [ ]:
#loop to align 6 transcripts from downloaded protein sequences
#this loop takes protein transcripts from ProteinBlast directory and outputs to that directory
#we then copy the .align file to the HMM directory for ease with HMM commands in the future

for i in ./ProteinBlast/*.fasta
do
../HMMandMuscle/muscle3.8.31_i86win32.exe -in $i -out $i.align
cp ./ProteinBlast/*.align ./HMM
done